In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!apt-get install unrar


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [ ]:
!unrar x "/content/drive/MyDrive/ML/Anomaly-Detection.rar" /content/Anomaly-Detection/


Streaming output truncated to the last 5000 lines.
Creating    /content/Anomaly-Detection/Anomaly-Detection/venv/Lib/site-packages/torch/include/google/protobuf/compiler/cpp  OK
Extracting  /content/Anomaly-Detection/Anomaly-Detection/venv/Lib/site-packages/torch/include/google/protobuf/compiler/cpp/cpp_generator.h      81%  OK 
Creating    /content/Anomaly-Detection/Anomaly-Detection/venv/Lib/site-packages/torch/include/google/protobuf/compiler/csharp  OK
Extracting  /content/Anomaly-Detection/Anomaly-Detection/venv/Lib/site-packages/torch/include/google/protobuf/compiler/csharp/csharp_generator.h      81%  OK 
Extracting  /content/Anomaly-Detection/Anomaly-Detection/venv/Lib/site-packages/torch/include/google/protobuf/compiler/csharp/csharp_names.h      81%  OK 
Extracting  /content/Anomaly-Detection/Anomaly-Detection/venv/Lib/site-packages/torch/include/google/protobuf/compiler/importer.h      81%  OK 
Creating    /content/Anomaly-Detection/Anomal

In [ ]:
%cd /content/Anomaly-Detection/Anomaly-Detection/


/content/Anomaly-Detection/Anomaly-Detection


In [ ]:
!ls


config.py    extract_features.py  model.py     requirements.txt  utils.py
data	     freia_funcs.py	  __pycache__  test.py		 venv
evaluate.py  main.py		  Readme.md    train.py


In [ ]:
!pip install -r requirements.txt

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16426 sha256=abc2a637b94f170813054e29df48f64d2f6a46f04400eb18ce6a11e782985ee6
  Stored in directory: /root/.cache/pip/wheels/9c/3f/43/e6271c7026fe08c185da2be23c98c8e87477d3db63f41f32ad
Successfully built efficientnet-pytorch


In [ ]:
!python main.py

Training Start...
209
Extractor skipped

Train epoch 0
Epoch: 0.0 	 train loss: 11.8267
Epoch: 0.4 	 train loss: 5.5871
Epoch: 0.8 	 train loss: 2.0026
Epoch: 0.12 	 train loss: 1.7000
Epoch: 0.16 	 train loss: 1.5870
Epoch: 0.20 	 train loss: 1.5566
Epoch: 0.24 	 train loss: 1.4832
Epoch: 0.28 	 train loss: 1.4126
Epoch: 0.32 	 train loss: 1.4160
Epoch: 0.36 	 train loss: 1.3362
Epoch: 0.40 	 train loss: 1.2427
Epoch: 0.44 	 train loss: 1.1847
Epoch: 0.48 	 train loss: 1.0536
Epoch: 0.52 	 train loss: 0.9614
Epoch: 0.56 	 train loss: 0.8660

Compute loss and scores on test set:
note..
note..
note..
note..
note..
note..
Epoch: 0 	 test_loss: 1.4296
AUROC: 	 last: 0.9976 	 max: 0.9976 	 epoch_max: 0 	 epoch_loss: 0

Train epoch 1

Compute loss and scores on test set:
note..
note..
note..
note..
note..
note..
Epoch: 1 	 test_loss: 1.3400
AUROC: 	 last: 0.9944 	 max: 0.9976 	 epoch_max: 0 	 epoch_loss: 0

Train epoch 2

Compute loss and scores on test set:
note..
note..
note..
note..
note

In [ ]:
from google.colab import files
files.download('/content/Anomaly-Detection/Anomaly-Detection/models/tmp/bottle_test')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!ls /content/Anomaly-Detection/Anomaly-Detection/models/tmp/bottle_test


/content/Anomaly-Detection/Anomaly-Detection/models/tmp/bottle_test


In [ ]:
import sys
sys.path.append('/content/Anomaly-Detection/Anomaly-Detection')

import torch
from freia_funcs import ReversibleGraphNet
from model import CrossConvolutions, parallel_glow_coupling_layer
from utils import FeatureDataset, preprocess_batch
import config as c
import matplotlib.pyplot as plt

# -------------------------
# Step 1: Load model on CPU (avoid CUDA errors)
# -------------------------
checkpoint_path = "/content/Anomaly-Detection/Anomaly-Detection/models/tmp/bottle_test"

# safe loading
model = torch.load(checkpoint_path, map_location='cpu')
model.eval()
c.device = 'cpu'
model.to(c.device)
print("✅ Model loaded successfully on CPU!")

# -------------------------
# Step 2: Load pre-extracted feature maps for test image(s)
# -------------------------
# Assume features already exist as .npy files in data/features/<class_name>/
test_dataset = FeatureDataset(train=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

# -------------------------
# Step 3: Run inference on one example
# -------------------------
for i, (feature_maps, label) in enumerate(test_loader):
    # Move features to device
    feature_maps = [fmap.to(c.device) for fmap in feature_maps]

    with torch.no_grad():
        output = model(feature_maps)

    # Convert output to numpy for visualization
    if isinstance(output, list):
        anomaly_map = output[0].squeeze().cpu().numpy()
    else:
        anomaly_map = output.squeeze().cpu().numpy()

    plt.imshow(anomaly_map, cmap='jet')
    plt.colorbar()
    plt.title(f"Detected defects / anomaly map (label={label.item()})")
    plt.show()

    break  # only first sample


UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL freia_funcs.ReversibleGraphNet was not an allowed global by default. Please use `torch.serialization.add_safe_globals([freia_funcs.ReversibleGraphNet])` or the `torch.serialization.safe_globals([freia_funcs.ReversibleGraphNet])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to lea